### Setup some basic stuff

In [1]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
import folium
import branca.element as bre
import json
import geojson as gj

In [3]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [4]:
def get_row_count(n_maps, cols):
    rows = (n_maps / cols)
    if (n_maps % cols != 0):
        rows = rows + 1
    return rows

### Read the data

In [5]:
spec_to_validate = json.load(open("evaluation.spec.filled.json"))
sensing_configs = json.load(open("sensing_regimes.all.specs.json"))

### Validating calibration trips

In [6]:
def get_map_for_calibration_test(trip):
    curr_map = folium.Map()
    if trip["start_loc"] is None or trip["end_loc"] is None:
        return curr_map
    curr_start = lonlat_swap(trip["start_loc"]["coordinates"])
    curr_end = lonlat_swap(trip["end_loc"]["coordinates"])
    folium.Marker(curr_start, icon=folium.Icon(color="green"),
                  popup="Start: %s" % trip["start_loc"]["name"]).add_to(curr_map)
    folium.Marker(curr_end, icon=folium.Icon(color="red"),
                  popup="End: %s" % trip["end_loc"]["name"]).add_to(curr_map)
    folium.PolyLine([curr_start, curr_end], popup=trip["id"]).add_to(curr_map)
    curr_map.fit_bounds([curr_start, curr_end])    
    return curr_map

In [7]:
calibration_tests = spec_to_validate["calibration_tests"]
rows = get_row_count(len(calibration_tests), 4)
calibration_maps = bre.Figure((rows,4))
for i, t in enumerate(calibration_tests):
    if t["config"]["sensing_config"] != sensing_configs[t["config"]["id"]]["sensing_config"]:
        print("Mismatch in config for test" % t)
    curr_map = get_map_for_calibration_test(t)
    calibration_maps.add_subplot(rows, 4, i+1).add_child(curr_map)
calibration_maps

### Validating evaluation trips

In [8]:
def get_map_for_evaluation_trip(trip):
    curr_map = folium.Map()
    curr_start = lonlat_swap(trip["start_loc"]["coordinates"])
    curr_end = lonlat_swap(trip["end_loc"]["coordinates"])
    folium.Marker(curr_start, icon=folium.Icon(color="green"),
                  popup="Start: %s" % trip["start_loc"]["name"]).add_to(curr_map)
    folium.Marker(curr_end, icon=folium.Icon(color="red"),
                  popup="End: %s" % trip["end_loc"]["name"]).add_to(curr_map)
    latlng_route_coords = [lonlat_swap(rc) for rc in trip["route_coords"]]
    folium.PolyLine(latlng_route_coords,
                    popup="%s: %s" % (trip["mode"], trip["name"])).add_to(curr_map)
    for i, c in enumerate(latlng_route_coords):
        folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
    curr_map.fit_bounds([curr_start, curr_end])    
    return curr_map

In [9]:
evaluation_trips = spec_to_validate["evaluation_trips"]
rows = get_row_count(len(evaluation_trips), 2)
evaluation_maps = bre.Figure()
for i, t in enumerate(evaluation_trips):
    curr_map = get_map_for_evaluation_trip(t)
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)
evaluation_maps

### Validating sensing settings

In [10]:
for ss in spec_to_validate["sensing_settings"]:
    compare_list = ss["compare"]
    for i, ssc in enumerate(ss["sensing_configs"]):
        if ssc["id"] != compare_list[i]:
            print("Mismatch in sensing configurations for %s" % ss)